In [2]:
import os
from pathlib import Path
import pandas as pd

from sklearn.datasets import load_diabetes

from datahandles import preprocess_numeric
from utils import Config

from tabllm import load_dataset, load_and_preprocess_dataset
from datasets import load_from_disk

import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
ds_name = 'heart'
ds_raw = load_dataset(dataset_name=ds_name, data_dir=Path(f'./tabllm/data/datasets/{ds_name}'))
print(ds_raw)
print(ds_raw.columns)

In [ ]:
all_cols = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety_dict']
categorical = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety_dict']
numerical = list(set(all_cols) - set(categorical) - set(['label']))

print(categorical)
print(numerical)

In [ ]:
ds_name = 'income'
ds_raw = load_dataset(dataset_name=ds_name, data_dir=Path(f'./tabllm/data/datasets/{ds_name}'))
# print(ds_raw)
ds_txt = load_from_disk(f"tabllm/data/datasets_serialized/{ds_name}")
ds_txt = pd.DataFrame(ds_txt) # type: ignore
val_counts = ds_raw['label'].value_counts()
print(val_counts)
print('Max const. predictor acc:', max(val_counts)/sum(val_counts) * 100)

label
False    19694
True      6354
Name: count, dtype: int64
Max const. predictor acc: 75.60657248157248


In [8]:
ds_name = "income"
# ds = load_from_disk(f"tabllm/data/data_ser/{ds_name}")
ds = load_dataset(dataset_name=ds_name, data_dir=Path(f"./tabllm/data/datasets/{ds_name}"))
print(ds)

       age         workclass     education      marital_status  \
16465   39  Self-emp-not-inc          11th  Married-civ-spouse   
5625    54  Self-emp-not-inc     Bachelors  Married-civ-spouse   
30273   32           Private       HS-grad  Married-civ-spouse   
3136    45  Self-emp-not-inc  Some-college       Never-married   
4521    60           Private          10th  Married-civ-spouse   
...    ...               ...           ...                 ...   
32511   25         Local-gov     Bachelors       Never-married   
5192    32           Private     Bachelors  Married-civ-spouse   
12172   27           Private     Bachelors       Never-married   
235     59         State-gov       HS-grad  Married-civ-spouse   
29733   33           Private     Bachelors  Married-civ-spouse   

              occupation   relationship                race     sex  \
16465   Transport-moving        Husband               White    Male   
5625     Exec-managerial        Husband               White    Ma

In [11]:
from transformers import BertModel, BertTokenizer

model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [12]:
for name, param in model.named_parameters():
    print(name)
    # if "classifier" not in name: # Assuming the last layer is named "classifier"
    #     param.requires_grad = False

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

In [23]:
from utils import Config, ConfigObject
from datahandles import TabLLMDataObject, CombinedTabLLMTextDataset
import yaml
from jinja2 import Environment, FileSystemLoader, select_autoescape
import os

cfg = Config("./cfgs/bert.yaml")
tabllm_do = TabLLMDataObject(cfg, set_hyponet_in_dim=True)
test_ds = tabllm_do.data['test']
print(test_ds[0]['shots'])

TabLLMDataObject initialized with 4 training set(s),1 validation set(s), and 1 test set(s).
Maximum number of features across all datasets: 103
Hyponet in_dim set to max number of features (=103)
[(48842, 48842), (748, 748), (45211, 45211), (20640, 20640)]
Example: The median income is 2.1516. The median age is 52. The total rooms is 1254. The total bedrooms is 469. The population is 895. The households is 456. The latitude is 37.78. The longitude is -122.42. The label is True.

  Example: The median income is 2.6458. The median age is 50. The total rooms is 629. The total bedrooms is 188. The population is 742. The households is 196. The latitude is 37.79. The longitude is -122.25. The label is False.

  Example: The median income is 5.7051. The median age is 35. The total rooms is 2142. The total bedrooms is 373. The population is 986. The households is 374. The latitude is 34.17. The longitude is -118.07. The label is True.

 


In [ ]:
class CustomTagLoader(yaml.SafeLoader):
    pass

# Add constructors for the specific tags you want to treat as regular mappings.
# This tells PyYAML to parse the content under these tags as a standard dictionary.
CustomTagLoader.add_constructor(u'!Template', CustomTagLoader.construct_mapping)
CustomTagLoader.add_constructor(u'!TemplateMetadata', CustomTagLoader.construct_mapping)


def load_and_apply_template(file_path: str, note: str, label: int) -> str:
    """
    Loads a YAML template from a file, applies the given note and label using Jinja2.

    Args:
        file_path (str): The path to the YAML template file.
        note (str): The note text to be inserted into the template.
        label (int): The index of the answer choice to be selected (0 for 'No', 1 for 'Yes').

    Returns:
        str: The rendered string from the Jinja template.
    """
    try:
        # Load the YAML content from the file using the custom loader
        with open(file_path, 'r') as f:
            template_data = yaml.load(f, Loader=CustomTagLoader)

        # Extract the relevant template information
        # Assuming the structure is dataset -> templates -> <uuid> -> jinja
        # And answer_choices is also under the <uuid>
        template_id = list(template_data['templates'].keys())[0] # Get the first (and likely only) template ID
        template_config = template_data['templates'][template_id]

        jinja_string = template_config['jinja']
        answer_choices_raw = template_config['answer_choices']

        # Split answer choices into a list
        answer_choices = [choice.strip() for choice in answer_choices_raw.split('|||')]

        # Set up Jinja2 environment (no specific loader needed as we have the string directly)
        env = Environment(
            loader=FileSystemLoader(os.path.dirname(file_path) or './'), # Use FileSystemLoader for relative includes, though not strictly needed here
            autoescape=select_autoescape(['html', 'xml'])
        )

        # Load the template from the string
        template = env.from_string(jinja_string)

        # Render the template with the provided data
        rendered_output = template.render(
            note=note,
            label=label,
            answer_choices=answer_choices # Pass the list of answer choices
        )

        return rendered_output

    except FileNotFoundError:
        return f"Error: File not found at {file_path}"
    except yaml.YAMLError as e:
        return f"Error parsing YAML file: {e}"
    except KeyError as e:
        return f"Error: Missing key in template structure: {e}. Please check the YAML format."
    except IndexError:
        return f"Error: Label {label} is out of bounds for answer choices {answer_choices}. Please provide a valid label index."
    except Exception as e:
        return f"An unexpected error occurred: {e}"

print(test_ds[0]['queries_y'])
out = load_and_apply_template("./tabllm/templates/templates_blood.yaml", note=test_ds[0]['shots'], label=test_ds[0]['queries_y'])
print(out)

[0 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0 1 0 1 1]
Example: The median income is 2.1516. The median age is 52. The total rooms is 1254. The total bedrooms is 469. The population is 895. The households is 456. The latitude is 37.78. The longitude is -122.42. The label is True.

  Example: The median income is 2.6458. The median age is 50. The total rooms is 629. The total bedrooms is 188. The population is 742. The households is 196. The latitude is 37.79. The longitude is -122.25. The label is False.

  Example: The median income is 5.7051. The median age is 35. The total rooms is 2142. The total bedrooms is 373. The population is 986. The households is 374. The latitude is 34.17. The longitude is -118.07. The label is True.

 

Did the person donate blood? Yes or no? Answer: ||| 


An unexpected error occurred: 'NoneType' object is not subscriptable


In [36]:
template_content = """
dataset: bank
templates:
  fc3d4b70-f59e-4fbb-9582-7e118946162c: !Template
    name: bank
    id: fc3d4b70-f59e-4fbb-9582-7e118946162c
    reference: ''
    answer_choices: 'No ||| Yes'
    jinja: '{{note}}
      
      
      Does this client subscribe to a term deposit? Yes or no?
      Answer: 
      |||
      {{ answer_choices[label] }}'
    metadata: !TemplateMetadata
      choices_in_prompt: true
      metrics: ['accuracy']
      original_task: true
"""
template_file_name = "templates_bank.yaml"
with open(template_file_name, "w") as f:
    f.write(template_content)

# Example 1: Label 'Yes'
sample_note_1 = "Client has a good credit score and recently opened a savings account."
sample_label_1 = 1 # Corresponds to 'Yes'
print("--- Example 1 (Label 'Yes') ---")
result_1 = load_and_apply_template(template_file_name, sample_note_1, sample_label_1)
print(result_1)
print("\n" + "="*50 + "\n")

# Example 2: Label 'No'
sample_note_2 = "Client has a history of missed payments and high debt."
sample_label_2 = 0 # Corresponds to 'No'
print("--- Example 2 (Label 'No') ---")
result_2 = load_and_apply_template(template_file_name, sample_note_2, sample_label_2)
print(result_2)
print("\n" + "="*50 + "\n")

# Example 3: Invalid file path
print("--- Example 3 (Invalid File) ---")
result_3 = load_and_apply_template("non_existent_file.yaml", "Some note", 0)
print(result_3)
print("\n" + "="*50 + "\n")

# Clean up the dummy file
# os.remove(template_file_name)

--- Example 1 (Label 'Yes') ---
{'dataset': 'bank', 'templates': {'fc3d4b70-f59e-4fbb-9582-7e118946162c': None}}
An unexpected error occurred: 'NoneType' object is not subscriptable


--- Example 2 (Label 'No') ---
{'dataset': 'bank', 'templates': {'fc3d4b70-f59e-4fbb-9582-7e118946162c': None}}
An unexpected error occurred: 'NoneType' object is not subscriptable


--- Example 3 (Invalid File) ---
Error: File not found at non_existent_file.yaml


